In [1]:
# Import required libraries 

import numpy as np
import random
import copy
from numba import jit
import time
import cProfile

In [2]:
class World:
    """
    The World class encapsulates information about the computational domain.
    
    Attributes
    ----------
    <to be written later>
    ----------
    
    """
    
    def __init__(self, ni, nj, nk):
        """
        Initializes the World object with the grid parameters
        ni, nj, nk

        Parameters
        ----------
        ni : int
            number of grid points along x coordinate
        nj : int
            number of grid points along y coordinate
        nk : int
            number of grid points along z coordinate
        ----------
        
        """
        
        self.ni = ni # number of grid points along x coordinate
        self.nj = nj # number of grid points along y coordinate
        self.nk = nk # number of grid points along z coordinate
        
        self.nn = np.zeros(3) # number of grid points along x,y,z coordinates
        
        self.nn[0] = self.ni # number of grid points along x coordinate
        self.nn[1] = self.nj # number of grid points along y coordinate
        self.nn[2] = self.nk # number of grid points along z coordinate
        
        self.x0 = np.zeros(3) # mesh origin vector
        self.dh = np.zeros(3) # mesh spacing vector
        self.xm = np.zeros(3) # mesh max bound vector
        self.xc = np.zeros(3) # mesh centroid
        
        self.EPS_0 = 8.85418782e-12    # epsilon_0
        self.QE    = 1.602176565e-19;  # proton charge
        self.AMU   = 1.660538921e-27   # amu
        self.ME    = 9.10938215e-31;   # mass of electron    
        self.K     = 1.380648e-23;     # Boltzmann's const.
        self.EvToK = self.QE/self.K;   # 1 eV = q/k
        
        self.phi        = np.zeros((self.ni, self.nj, self.nk)) # potential phi
        self.object_id  = np.zeros((self.ni, self.nj, self.nk)) # grid point classifier
        self.phi_new    = np.zeros((self.ni, self.nj, self.nk)) # array for phi_new
        self.R          = np.zeros((self.ni, self.nj, self.nk)) # residual R
        self.R1         = np.zeros((self.ni, self.nj, self.nk)) # residual R
        self.rho        = np.zeros((self.ni, self.nj, self.nk)) # rho
        self.node_vol   = np.zeros((self.ni, self.nj, self.nk)) # node volume
        self.ef         = np.zeros((self.ni, self.nj, self.nk, 3)) # electric field
    
    def setTime(self, dt, num_ts):
        """
        Set simulation timestep and total number of timesteps.
        
        Parameters
        ----------
        dt : float
            simulation timestep
        num_ts : int
            total number of timesteps
        ----------
        
        """
        self.dt     = dt     # timestep
        self.num_ts = num_ts # total number of timesteps
        
    def setExtents(self, x1, y1, z1, x2, y2, z2):
        
        """
        Set mesh extents and compute grid spacing.

        Parameters
        ----------
        x1 : float
            x-coordinate of world origin
        y1 : float
            y-coordinate of world origin
        z1 : float
            z-coordinate of world origin
        x2 : float
            x-coordinate of world max bound
        y2 : float
            y-coordinate of world max bound
        z2 : float
            z-coordinate of world max bound
        ----------
        
        """
        
        self.x0[0] = x1 # x-coordinate of world origin
        self.x0[1] = y1 # y-coordinate of world origin
        self.x0[2] = z1 # z-coordinate of world origin
        
        self.xm[0] = x2 #x-coordinate of world max bound
        self.xm[1] = y2 #y-coordinate of world max bound
        self.xm[2] = z2 #z-coordinate of world max bound 
        
        # Compute mesh spacing dh, and mesh centroid
        for i in range(3):
            self.dh[i] = (self.xm[i] - self.x0[i]) / (self.nn[i] - 1)
            self.xc[i] = 0.5*(self.x0[i] + self.xm[i]) 
        
        # Compute node volumes
        self.computeNodeVolumes()
        
    def addSphere(self, sphere_x0, sphere_radius, sphere_phi):
        """
        Add sphere to grid using sugarcube technique.

        Parameters
        ----------
        sphere_x0 : numpy.ndarray
            position vector of sphere origin (xs0, ys0, zs0)
        sphere_radius : float
            sphere radius
        sphere_phi : float
            sphere potential
        ----------
        
        """
        
        self.sphere_x0     = sphere_x0        # sphere origin
        self.sphere_radius = sphere_radius    # sphere radius
        self.sphere_radius2= sphere_radius**2 # sphere radius^2
        self.sphere_phi    = sphere_phi       # sphere potential

        # loop over all i, j, k

        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):

                    # check if pos(i,j,k) lies inside sphere
                    x = self.pos(np.array([i,j,k]))

                    if self.inSphere(x):
                        # if inside, set object_id[i,j,k] = 1,
                        # if inside, set phi[i,j,k] = self.sphere_phi
                        self.object_id[i,j,k] = 1
                        self.phi[i,j,k] = self.sphere_phi

    
    def addInlet(self, phi_inlet):
        """
        Add inlet to simulation domain at k = 0.

        Parameters
        ----------
        phi_inlet : float
            inlet potential
        ----------
        """
        self.phi_inlet = phi_inlet
        
        # loop over all i, j at k = 0
        for i in np.arange(0, self.ni):
                for j in np.arange(0, self.nj):
                    # set object_id[i,j,0] = 2
                    # set phi[i,j,0]       = 0
                    self.object_id[i,j,0] = 2
                    self.phi[i,j,0]       = 0
                    
    
    def markBoundaryCells(self):
        
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):
                    
                    if i==0 or i==self.ni-1 or j==0 or j==self.nj-1 or k==0 or k==self.nk-1:
                        self.object_id[i,j,k]=3
                
                
                    
        
                                       
    
    def computeDebyeLength(self, Te, ne):
        """
        Compute the Debye length.

        Parameters
        ----------
        Te : float
            electron temperature, K
        ne : float
            number density, m3
        ----------
        
        """
        
        return np.sqrt(self.EPS_0*self.K*Te/(ne*self.QE**2))
    
    def potentialSolver6(self, w, max_it, tol):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # Step 1: create *integer* array the same size as u 
        x = np.zeros_like(self.phi,dtype=np.int)

        # Step 2: populate all non-boundary cells with running numbers from 1 to (n-2)^2
        x[1:-1,1:-1,1:-1] = np.arange(1,(self.ni-2)*(self.nj-2)*(self.nk-2)+1).reshape(self.ni-2,self.nj-2,self.nk-2)

        # Step 3: get indices of even (red) and odd (black) points
        ir, jr, kr = np.where((x>0) & (x%2 == 0)) # indices of red pts = indices of even numbers
        ib, jb, kb = np.where((x>0) & (x%2 == 1)) # indices of black pts = indices of odd numbers

        # solve potential
        for it in np.arange(1,max_it+1):
            """
            for i in np.arange(1,self.ni-1):
                for j in np.arange(1,self.nj-1):
                    for k in np.arange(1,self.nk-1):
            """            
            """
            #standard internal open node
                        phi_new = (self.rho[i][j][k]/self.EPS_0 +\
                                        dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                                        dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                                        dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1]))/(2*dx2+2*dy2+2*dz2)
                                        
            # sucessive over relaxation  
                        self.phi[i,j,k] = self.phi[i,j,k] + 1.4*(phi_new - self.phi[i][j][k])
        
            """ 
            
            """
            #standard internal open node
            self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                            (self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                            dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1])+\
                            dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1])+\
                            dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk]))/(2*dx2+2*dy2+2*dz2)
            
            """
            """
            # Red point update
            self.phi[ir,jr,kr] = (1-w)*self.phi[ir,jr,kr] + (1.0/6.0)*w*(self.phi[ir+1,jr,kr] + self.phi[ir-1,jr,kr] +\
                                                                    self.phi[ir,jr+1,kr] + self.phi[ir,jr-1,kr] +\
                                                                    self.phi[ir,jr,kr+1] + self.phi[ir,jr,kr-1] +\
                                                                    (self.rho[ir,jr,kr]/self.EPS_0)*(self.dh[0]*self.dh[1]))

            # Black point update
            self.phi[ib,jb,kb] = (1-w)*self.phi[ib,jb,kb] + (1.0/6.0)*w*(self.phi[ib+1,jb,kb] + self.phi[ib-1,jb,kb] +\
                                                                    self.phi[ib,jb+1,kb] + self.phi[ib,jb-1,kb] +\
                                                                    self.phi[ib,jb,kb+1] + self.phi[ib,jb,kb-1] +\
                                                                    (self.rho[ib,jb,kb]/self.EPS_0)*(self.dh[0]*self.dh[1]))
            """
            
            # Red point update
            self.phi[ir,jr,kr] = (1-w)*self.phi[ir,jr,kr] + w*((self.phi[ir+1,jr,kr] + self.phi[ir-1,jr,kr])*dx2 +\
                                                                         (self.phi[ir,jr+1,kr] + self.phi[ir,jr-1,kr])*dy2 +\
                                                                         (self.phi[ir,jr,kr+1] + self.phi[ir,jr,kr-1])*dz2 +\
                                                                         (self.rho[ir,jr,kr]/self.EPS_0))/(2*dx2+2*dy2+2*dz2)

            # Black point update
            self.phi[ib,jb,kb] = (1-w)*self.phi[ib,jb,kb] + w*((self.phi[ib+1,jb,kb] + self.phi[ib-1,jb,kb])*dx2 +\
                                                                         (self.phi[ib,jb+1,kb] + self.phi[ib,jb-1,kb])*dy2 +\
                                                                         (self.phi[ib,jb,kb+1] + self.phi[ib,jb,kb-1])*dz2 +\
                                                                         (self.rho[ib,jb,kb]/self.EPS_0))/(2*dx2+2*dy2+2*dz2)
           
            #check for convergence*/
            if it%25==0:
                sum = 0;
                """
                for i in np.arange(1,self.ni-1):
                    for j in np.arange(1,self.nj-1):
                        for k in np.arange(1,self.nk-1):
                """

                self.R[1:self.ni-1,1:self.nj-1,1:self.nk-1] = \
                -self.phi[1:self.ni-1,1:self.nj-1,1:self.nk-1]*(2*dx2+2*dy2+2*dz2) +\
                self.rho[1:self.ni-1,1:self.nj-1,1:self.nk-1]/self.EPS_0 +\
                dx2*(self.phi[0:self.ni-2,1:self.nj-1,1:self.nk-1] + self.phi[2:self.ni,1:self.nj-1,1:self.nk-1]) +\
                dy2*(self.phi[1:self.ni-1,0:self.nj-2,1:self.nk-1] + self.phi[1:self.ni-1,2:self.nj,1:self.nk-1]) +\
                dz2*(self.phi[1:self.ni-1,1:self.nj-1,0:self.nk-2] + self.phi[1:self.ni-1,1:self.nj-1,2:self.nk])

                sum = np.sum(self.R**2)

                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))
                if (L2<tol):
                    converged = True
                    break

        #if (converged==False):
        #    print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged
    
    def potentialSolver7(self, w, max_it, tol, n0, phi0, Te0):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # solve potential
        for it in np.arange(1,max_it+1):
            for i in np.arange(0,self.ni):
                for j in np.arange(0,self.nj):
                    for k in np.arange(0,self.nk):
                        if self.object_id[i,j,k] > 0 :
                            # skip over solid (fixed) nodes
                            continue
                        if (i==0):
                            self.phi[i,j,k] = self.phi[i+1,j,k]
                        elif (i==self.ni-1):
                            self.phi[i,j,k] = self.phi[i-1,j,k]
                        elif (j==0):
                            self.phi[i,j,k] = self.phi[i,j+1,k]
                        elif (j==self.nj-1):
                            self.phi[i,j,k] = self.phi[i,j-1,k]
                        elif (k==0):
                            self.phi[i,j,k] = self.phi[i,j,k+1]
                        elif (k==self.nk-1):
                            self.phi[i,j,k] = self.phi[i,j,k-1]
                        else:
                            # standard internal node
                            ne = n0*np.exp((self.phi[i][j][k]-phi0)/Te0)
                            
                            self.phi[i][j][k] = ((self.rho[i][j][k]-self.QE*ne)/self.EPS_0 +
                                        dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +
                                        dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +
                                        dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1]))/(2*dx2+2*dy2+2*dz2)
                            
                            # SOR
                            #self.phi[i][j][k] = self.phi[i][j][k] + w*(phi_new-self.phi[i][j][k])
                            
                            
            #check for convergence*/
            if it%25==0:
                sum = 0;
                for i in np.arange(0,self.ni):
                    for j in np.arange(0,self.nj):
                        for k in np.arange(0,self.nk):
                            if self.object_id[i,j,k] > 0 :
                                # skip over solid (fixed) nodes
                                continue
                            
                            R = 0.0
                            
                            if (i==0):
                                R = self.phi[i,j,k] - self.phi[i+1,j,k]
                            elif (i==self.ni-1):
                                R = self.phi[i,j,k] - self.phi[i-1,j,k]
                            elif (j==0):
                                R = self.phi[i,j,k] - self.phi[i,j+1,k]
                            elif (j==self.nj-1):
                                R = self.phi[i,j,k] - self.phi[i,j-1,k]
                            elif (k==0):
                                R = self.phi[i,j,k] - self.phi[i,j,k+1]
                            elif (k==self.nk-1):
                                R = self.phi[i,j,k] - self.phi[i,j,k-1]
                            
                            else:
                                ne = n0*np.exp((self.phi[i][j][k]-phi0)/Te0)
 
                                R = -self.phi[i][j][k]*(2*dx2+2*dy2+2*dz2) +\
                                (self.rho[i][j][k]-self.QE*ne)/self.EPS_0 +\
                                dx2*(self.phi[i-1][j][k] + self.phi[i+1][j][k]) +\
                                dy2*(self.phi[i][j-1][k] + self.phi[i][j+1][k]) +\
                                dz2*(self.phi[i][j][k-1] + self.phi[i][j][k+1])

                            sum += R*R;


                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                print("iter: "+str(it)+", L2 = "+str(L2))
                if (L2<tol):
                    converged = True
                    break
                    
        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged
    
    
    def potentialSolver8(self, w, max_it, tol, n0, phi0, Te0):
        """
        Compute the potential field.

        Parameters
        ----------
        max_it : int
            max iterations for Gauss-Seidel
        tol: float
            tolerance for Gauss-Seidel
        ----------
        
        """

        dx2 = 1.0/(self.dh[0]*self.dh[0]); # dx^2
        dy2 = 1.0/(self.dh[1]*self.dh[1]); # dy^2
        dz2 = 1.0/(self.dh[2]*self.dh[2]); # dz^2
    
        L2 = 0.0 # norm
        
        converged = False
        
        # solve potential
        #for it in np.arange(1,max_it+1):
        #    for i in np.arange(0,self.ni):
        #        for j in np.arange(0,self.nj):
        #            for k in np.arange(0,self.nk):
        #if self.object_id[i,j,k] > 0 :
        #    # skip over solid (fixed) nodes
        #    continue
        
        
        #if (i==0):
        #    self.phi[i,j,k] = self.phi[i+1,j,k]
        #elif (i==self.ni-1):
        #    self.phi[i,j,k] = self.phi[i-1,j,k]
        #elif (j==0):
        #    self.phi[i,j,k] = self.phi[i,j+1,k]
        #elif (j==self.nj-1):
        #    self.phi[i,j,k] = self.phi[i,j-1,k]
        #elif (k==0):
        #    self.phi[i,j,k] = self.phi[i,j,k+1]
        #elif (k==self.nk-1):
        #    self.phi[i,j,k] = self.phi[i,j,k-1]
        
        x    = np.where(self.object_id == 0)
        xarr = np.asarray(x)
        u    = xarr.T
        
        for it in np.arange(1,max_it+1):
        
            self.phi[0,:,1:]         = self.phi[1,:,1:]
            self.phi[self.ni-1,:,1:]  = self.phi[self.ni-2,:,1:]

            self.phi[:,0,1:]         = self.phi[:,1,1:]
            self.phi[:,self.nj-1,1:]  = self.phi[:,self.nj-2,1:]

            #self.phi[:,:,0]         = self.phi[:,:,1]
            self.phi[:,:,self.nk-1] = self.phi[:,:,self.nk-2]

            # standard internal node
          
            ne = np.zeros((self.ni, self.nj, self.nk)) 
            ne[u[:,0], u[:,1], u[:,2]] = n0*np.exp((self.phi[u[:,0], u[:,1], u[:,2]]-phi0)/Te0)

            #phi_new np.zeros(len(xarr[0,:]))

            self.phi[u[:,0], u[:,1], u[:,2]] = ((self.rho[u[:,0], u[:,1], u[:,2]]-self.QE*ne[u[:,0], u[:,1], u[:,2]])/self.EPS_0 +
                        dx2*(self.phi[u[:,0]-1, u[:,1], u[:,2]] + self.phi[u[:,0]+1, u[:,1], u[:,2]]) +
                        dy2*(self.phi[u[:,0], u[:,1]-1, u[:,2]] + self.phi[u[:,0], u[:,1]+1, u[:,2]]) +
                        dz2*(self.phi[u[:,0], u[:,1], u[:,2]-1] + self.phi[u[:,0], u[:,1], u[:,2]+1]))/(2*dx2+2*dy2+2*dz2)

            
                          
            #check for convergence*/
            if it%25==0:
                sum = 0;
                #for i in np.arange(0,self.ni):
                #    for j in np.arange(0,self.nj):
                #        for k in np.arange(0,self.nk):
                #            if self.object_id[i,j,k] > 0 :
                #                # skip over solid (fixed) nodes
                #                continue
                            

                
                self.R[0,:,1:]         = self.phi[0,:,1:] - self.phi[1,:,1:]
                self.R[self.ni-1,:,1:] = self.phi[self.ni-1,:,1:] - self.phi[self.ni-2,:,1:]
                
                self.R[:,0,1:]         = self.phi[:,0,1:] - self.phi[:,1,1:]
                self.R[:,self.nj-1,1:] = self.phi[:,self.nj-1,1:] - self.phi[:,self.nj-2,1:]
                
                #self.R[:,:,0]         = self.phi[:,:,0] - self.phi[:,:,1]
                self.R[:,:,self.nk-1] = self.phi[:,:,self.nk-1] - self.phi[:,:,self.nk-2]


                ne = np.zeros((self.ni, self.nj, self.nk)) 
                ne[u[:,0], u[:,1], u[:,2]] = n0*np.exp((self.phi[u[:,0], u[:,1], u[:,2]]-phi0)/Te0)

                self.R[u[:,0], u[:,1], u[:,2]] = -self.phi[u[:,0], u[:,1], u[:,2]]*(2*dx2+2*dy2+2*dz2) +\
                (self.rho[u[:,0], u[:,1], u[:,2]]-self.QE*ne[u[:,0], u[:,1], u[:,2]])/self.EPS_0 +\
                dx2*(self.phi[u[:,0]-1, u[:,1], u[:,2]] + self.phi[u[:,0]+1, u[:,1], u[:,2]]) +\
                dy2*(self.phi[u[:,0], u[:,1]-1, u[:,2]] + self.phi[u[:,0], u[:,1]+1, u[:,2]]) +\
                dz2*(self.phi[u[:,0], u[:,1], u[:,2]-1] + self.phi[u[:,0], u[:,1], u[:,2]+1])

                sum = np.sum(self.R**2)
            
            
                L2 = np.sqrt(sum/(self.ni*self.nj*self.nk));
                #print("iter: "+str(it)+", L2 = "+str(L2))

                if (L2<tol):
                    converged = True
                    break

        if (converged==False):
            print("Gauss-Seidel failed to converge, L2 = "+str(L2))
        
        return converged
    
    
    def efSolver(self):
        """
        Compute the electric field from potential function.

        Parameters
        ----------
        
        ----------
        
        """
        dx = self.dh[0] # dx
        dy = self.dh[1] # dy
        dz = self.dh[2] # dz
        
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):

                    #x-component#
                    if i==0: 
                        # forward
                        self.ef[i][j][k][0] = -(-3*self.phi[i][j][k]+\
                                               4*self.phi[i+1][j][k]-\
                                               self.phi[i+2][j][k])/(2*dx)
                    elif i==self.ni-1:  
                        # backward
                        self.ef[i][j][k][0] = -(self.phi[i-2][j][k]-\
                                               4*self.phi[i-1][j][k]+\
                                               3*self.phi[i][j][k])/(2*dx)
                    else: 
                        #central
                        self.ef[i][j][k][0] = -(self.phi[i+1][j][k] - \
                                                self.phi[i-1][j][k])/(2*dx)

                    #y-component
                    if j==0:
                        self.ef[i][j][k][1] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j+1][k]-\
                                                self.phi[i][j+2][k])/(2*dy)
                    elif j==self.nj-1:
                        self.ef[i][j][k][1] = -(self.phi[i][j-2][k] - \
                                                4*self.phi[i][j-1][k] +\
                                                3*self.phi[i][j][k])/(2*dy)
                    else:
                         self.ef[i][j][k][1] = -(self.phi[i][j+1][k] - \
                                                 self.phi[i][j-1][k])/(2*dy)

                    #z-component
                    if k==0:
                        self.ef[i][j][k][2] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j][k+1]-
                                                self.phi[i][j][k+2])/(2*dz)
                    elif k==self.nk-1:
                        self.ef[i][j][k][2] = -(self.phi[i][j][k-2] - \
                                                4*self.phi[i][j][k-1]    + \
                                                3*self.phi[i][j][k])/(2*dz)
                    else:
                        self.ef[i][j][k][2] = -(self.phi[i][j][k+1] - \
                                                self.phi[i][j][k-1])/(2*dz)
    
    def efSolver2(self):
        """
        Compute the electric field from potential function.

        Parameters
        ----------
        
        ----------
        
        """
        dx = self.dh[0] # dx
        dy = self.dh[1] # dy
        dz = self.dh[2] # dz
        
        """
        for i in np.arange(0, self.ni):
            for j in np.arange(0, self.nj):
                for k in np.arange(0, self.nk):
        """

        ##x-component#
        #if i==0: 
        #x-component#
        """
                    if i==0: 
                        # forward
                        self.ef[i][j][k][0] = -(-3*self.phi[i][j][k]+\
                                               4*self.phi[i+1][j][k]-\
                                               self.phi[i+2][j][k])/(2*dx)
        """
                    
        # forward
        self.ef[0,0:self.nj,0:self.nk,0] = -(-3*self.phi[0,0:self.nj,0:self.nk]+\
                               4*self.phi[1,0:self.nj,0:self.nk]-\
                               self.phi[2,0:self.nj,0:self.nk])/(2*dx)
        
        #elif i==self.ni-1:  
        """
        elif i==self.ni-1:  
                        # backward
                        self.ef[i][j][k][0] = -(self.phi[i-2][j][k]-\
                                               4*self.phi[i-1][j][k]+\
                                               3*self.phi[i][j][k])/(2*dx)
        """           
        # backward
        self.ef[self.ni-1,0:self.nj,0:self.nk,0] = -(self.phi[self.ni-3,0:self.nj,0:self.nk]-\
                                   4*self.phi[self.ni-2,0:self.nj,0:self.nk]+\
                                   3*self.phi[self.ni-1,0:self.nj,0:self.nk])/(2*dx)
        """
        else: 
            #central
            self.ef[i][j][k][0] = -(self.phi[i+1][j][k] - \
                                    self.phi[i-1][j][k])/(2*dx)
        """ 
        #central
        self.ef[1:self.ni-1,0:self.nj,0:self.nk,0] = -(self.phi[2:self.ni,0:self.nj,0:self.nk] - \
                                self.phi[0:self.ni-2,0:self.nj,0:self.nk])/(2*dx)


        #y-component
        #if j==0:
        """
        if j==0:
                        self.ef[i][j][k][1] = -(-3*self.phi[i][j][k] + \
                                                4*self.phi[i][j+1][k]-\
                                                self.phi[i][j+2][k])/(2*dy)
                    
        """
        self.ef[0:self.ni,0,0:self.nk,1] = -(-3*self.phi[0:self.ni,0,0:self.nk] + \
                                    4*self.phi[0:self.ni,1,0:self.nk]-\
                                    self.phi[0:self.ni,2,0:self.nk])/(2*dy)
        #elif j==self.nj-1:
        """
        elif j==self.nj-1:
                        self.ef[i][j][k][1] = -(self.phi[i][j-2][k] - \
                                                4*self.phi[i][j-1][k] +\
                                                3*self.phi[i][j][k])/(2*dy)
                    
        """
        self.ef[0:self.ni,self.nj-1,0:self.nk,1] = -(self.phi[0:self.ni,self.nj-3,0:self.nk] - \
                                    4*self.phi[0:self.ni,self.nj-2,0:self.nk] +\
                                    3*self.phi[0:self.ni,self.nj-1,0:self.nk])/(2*dy)
        #else:
        """
        else:
                         self.ef[i][j][k][1] = -(self.phi[i][j+1][k] - \
                                                 self.phi[i][j-1][k])/(2*dy)

        """
        self.ef[0:self.ni,1:self.nj-1,0:self.nk,1] = -(self.phi[0:self.ni,2:self.nj,0:self.nk] - \
                                     self.phi[0:self.ni,0:self.nj-2,0:self.nk])/(2*dy)

        #z-component
        '''
        if k==0:
            self.ef[i][j][k][2] = -(-3*self.phi[i][j][k] + \
                                    4*self.phi[i][j][k+1]-
                                    self.phi[i][j][k+2])/(2*dz)
            
        '''
        #z-component
        #if k==0:
        self.ef[0:self.ni,0:self.nj,0,2] = -(-3*self.phi[0:self.ni,0:self.nj,0] + \
                                4*self.phi[0:self.ni,0:self.nj,1]-
                                self.phi[0:self.ni,0:self.nj,2])/(2*dz)

        """
        elif k==self.nk-1:
            self.ef[i][j][k][2] = -(self.phi[i][j][k-2] - \
                                    4*self.phi[i][j][k-1]    + \
                                    3*self.phi[i][j][k])/(2*dz)
        """
        
        #elif k==self.nk-1:
        self.ef[0:self.ni,0:self.nj,self.nk-1,2] = -(self.phi[0:self.ni,0:self.nj,self.nk-3] - \
                                    4*self.phi[0:self.ni,0:self.nj,self.nk-2]    + \
                                    3*self.phi[0:self.ni,0:self.nj,self.nk-1])/(2*dz) 
        """
        else:
            self.ef[i][j][k][2] = -(self.phi[i][j][k+1] - \
                                    self.phi[i][j][k-1])/(2*dz)
        """
        #else:
        self.ef[0:self.ni,0:self.nj,1:self.nk-1,2] = -(self.phi[0:self.ni,0:self.nj,2:self.nk] - \
                                    self.phi[0:self.ni,0:self.nj,0:self.nk-2])/(2*dz)
        
        
    def computeNodeVolumes(self): 
        """
        Compute the node volumes.
        Parameters
        ----------
        
        ----------
        
        """
        for i in np.arange(0,self.ni):
            for j in np.arange(0,self.nj):
                for k in np.arange(0,self.nk):
        
                    V = self.dh[0]*self.dh[1]*self.dh[2]
                    if (i==0 or i==self.ni-1): V*=0.5
                    if (j==0 or j==self.nj-1): V*=0.5
                    if (k==0 or k==self.nk-1): V*=0.5
                    
                    self.node_vol[i][j][k] = V
                    
    def XtoL(self, x):
        """
        Determine which cell a particle at position vector 
        x belongs to.
        
        Parameters
        ----------
        x : numpy.ndarray
            position vector 
        ----------
        """
        lc = np.zeros(3)
        
        lc[0] = (x[0]-self.x0[0])/self.dh[0];
        lc[1] = (x[1]-self.x0[1])/self.dh[1];
        lc[2] = (x[2]-self.x0[2])/self.dh[2];
        
        return lc
    
    def pos(self, lc):
        """
        Determine which position from logical coordinate
        
        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        ----------
        """
        x = np.zeros(3)
        
        x[0] = self.x0[0] + self.dh[0]*lc[0]
        x[1] = self.x0[1] + self.dh[1]*lc[1]
        x[2] = self.x0[2] + self.dh[2]*lc[2]
        
        return x

    
    def inSphere(self, x):
        """
        Determine which position from logical coordinate
        
        Parameters
        ----------
        x_vec : numpy.ndarray
            position vector vector x_vec[0] = (x1[0], x1[1], x1[2])
        ----------
        """
        
        r  = x - self.sphere_x0
        r2 = r[0]*r[0] + r[1]*r[1] + r[2]*r[2]
        
        if r2 <= self.sphere_radius2:
            return True
        else:
            return False
        
    
    def inBounds(self, x):
        """
        Determine if position x is within mesh bounds
        
        Parameters
        ----------
        x_vec : numpy.ndarray
            position vector vector x_vec[0] = (x1[0], x1[1], x1[2])
        ----------
        """
        
        for i in range(0,3):
            if (x[i] < self.x0[i] or x[i] >= self.xm[i]):
                return False
        
        return True
        
        
    
    def XtoLvec(self, x_vec):
        """
        Determine which cell a particle at position vector 
        x belongs to.
        
        Parameters
        ----------
        x_vec : numpy.ndarray
            position vector vector x_vec[0] = (x1[0], x1[1], x1[2])
        ----------
        """
        lc = np.zeros((len(x_vec[:,0]),3))
        
        lc[:,0] = (x_vec[:,0]-self.x0[0])/self.dh[0];
        lc[:,1] = (x_vec[:,1]-self.x0[1])/self.dh[1];
        lc[:,2] = (x_vec[:,2]-self.x0[2])/self.dh[2];
        
        return lc
    
    def addSpeciesList(self, speciesList):
        self.speciesList = speciesList
    
    def computeChargeDensity(self):
        """
        Compute the charge density.
        
        Parameters
        ----------
     
        ----------
        """
        
        self.rho = np.zeros((self.ni, self.nj, self.nk))
        
        for species in self.speciesList:
            if species.charge!=0:
                self.rho += species.charge*species.den         

In [3]:
class Particle:
    """
    The Particle class encapsulates information about the particles 
    used in the simulation.
    
    Attributes
    ----------
    
    ----------
    
    """
    
    def __init__(self, pos, vel, mpw):
        """
        Initializes the Particle object with the position, 
        speed, and macrparticle weight.

        Parameters
        ----------
        pos : numpy.ndarray
            particle position vector
        vel : numpy.ndarray
            particle velocity vector
        mpw : float
            macroparticle weight  
        ----------
        
        """
        
        self.pos = pos
        self.vel = vel
        self.mpw = mpw
        

class Species:
    """
    The Species class encapsulates information about the species 
    used in the simulation.
    
    Attributes
    ----------
    
    ----------
    
    """
    def __init__(self, name, mass, charge, mpw0, worldObj):
        """
        Initializes the Species object with the name, mass,
        charge.

        Parameters
        ----------
        name : str
            species name
        mass : float
            species mass
        charge : float
            species charge
        mpw : float
            macroparticle weight  
        ----------

        """
        
        self.particleList = []
        
        self.name   = name
        self.mass   = mass
        self.charge = charge
        self.mpw0   = mpw0
        
        self.den    = np.zeros((worldObj.ni, worldObj.nj, worldObj.nk))
        
        self.worldObj = worldObj
        
    def ColdBeamSource(self, v_drift, den):
        """
        Cold beam source

        Parameters
        ----------
        v_drift : float
            drift speed
        den : float
            number density
        ----------
        
        """
        
        Lx = self.worldObj.dh[0]*(self.worldObj.ni-1)
        Ly = self.worldObj.dh[1]*(self.worldObj.nj-1)
        
        A   = Lx*Ly
        
        num_real = den*v_drift*A*self.worldObj.dt
        
        num_sim  = int(num_real / self.mpw0 + random.random())
        
        for i in range(0, num_sim):
            pos = np.zeros(3)
            vel = np.zeros(3)
            
            pos[0] = self.worldObj.x0[0] + random.random()*Lx
            pos[1] = self.worldObj.x0[1] + random.random()*Ly
            pos[2] = self.worldObj.x0[2]
            
            vel[0] = 0.0
            vel[1] = 0.0
            vel[2] = v_drift
            
            self.addParticle(pos,vel,self.mpw0)     
        
    
    def addParticle(self, pos, vel, mpw):
        """random.random()
        add a particle to particleList

         Parameters
        ----------
        pos : numpy.ndarray
            particle position vector
        vel : numpy.ndarray
            particle velocity vector
        mpw : float
            macroparticle weight  
        ----------
        
        """
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoL(pos)

        #electric field at particle position
        ef_part = self.gather_ef(lc)

        #rewind velocity back by 0.5*dt*ef
        vel -=  self.charge/self.mass*ef_part*(0.5*self.worldObj.dt);
        
        #add particle to list
        self.particleList.append(Particle(pos, vel, mpw))
        
        
    def loadParticlesBox(self, x1, x2, num_den, num_mp):
        """
        loads randomly distributed particles in a x1-x2 box 
        representing num_den number density

        Parameters
        ----------
        x1 : numpy.ndarray
            origin of bounding box
        x2 : numpy.ndarray
            max. bound corner of box
        num_den : float
            number density
        num_mp  : number of macroparticles
        ----------

        """
        box_vol  = (x2[0]-x1[0])*(x2[1]-x1[1])*(x2[2]-x1[2]) # box vol.
        num_real = num_den * box_vol;                   #number of real particles
        mpw      = num_real/num_mp;                     # macroparticle weight
        
        self.box_vol = box_vol
        self.num_real= num_real
        self.mpw     = mpw

        #load particles on an equally spaced grid
        for p in np.arange(0,num_mp+1):
            # sample random position
            pos = np.zeros(3)
            vel = np.zeros(3)

            #rnd = random.random()

            pos[0] = x1[0] + random.random()*(x2[0]-x1[0]);
            pos[1] = x1[1] + random.random()*(x2[1]-x1[1]);
            pos[2] = x1[2] + random.random()*(x2[2]-x1[2]);

            #set initial velocity
            vel[0] = 0.0
            vel[1] = 0.0
            vel[2] = 0.0

            self.addParticle(pos,vel,mpw); # add new particle
            
    def loadParticlesBoxQS(self, x1, x2, num_den, num_mp):
        """
        loads randomly distributed particles in a x1-x2 box 
        representing num_den number density

        Parameters
        ----------
        x1 : numpy.ndarray
            origin of bounding box
        x2 : numpy.ndarray
            max. bound corner of box
        num_den : float
            number density
        num_mp  : numpy.ndarray
        ----------

        """
        box_vol  = (x2[0]-x1[0])*(x2[1]-x1[1])*(x2[2]-x1[2]) # box vol.
        num_real = num_den * box_vol;                   #number of real particles
        num_mp_tot = (num_mp[0]-1)*(num_mp[1]-1)*(num_mp[2]-1)
        mpw      = num_real/num_mp_tot;                     # macroparticle weight
        
        self.box_vol = box_vol
        self.num_real= num_real
        self.mpw     = mpw
        
        di = (x2[0]-x1[0])/(num_mp[0]-1);
        dj = (x2[1]-x1[1])/(num_mp[1]-1);
        dk = (x2[2]-x1[2])/(num_mp[2]-1);

        #load particles on a equally spaced grid
        
        for i in np.arange(0,num_mp[0]):
            for j in np.arange(0,num_mp[1]):
                for k in np.arange(0,num_mp[2]):
                    
                    # sample random position
                    pos = np.zeros(3)
                    vel = np.zeros(3)
                    
                    pos[0] = x1[0] + i*di;
                    pos[1] = x1[1] + j*dj;
                    pos[2] = x1[2] + k*dk;

                    # shift particles on max faces back to the domain
                    if (pos[0]==x2[0]): pos[0]-=1e-4*di;
                    if (pos[1]==x2[1]): pos[1]-=1e-4*dj;
                    if (pos[2]==x2[2]): pos[2]-=1e-4*dk;

                    w = 1;
                    if (i==0 or i==num_mp[0]-1): w*=0.5
                    if (j==0 or j==num_mp[1]-1): w*=0.5
                    if (k==0 or k==num_mp[2]-1): w*=0.5

                    #set initial velocity
                    vel[0] = 0.0
                    vel[1] = 0.0
                    vel[2] = 0.0

                    self.addParticle(pos,vel,mpw*w)
                    
    
    def scatter_den(self, lc, value):
        """
        scatters scalar value onto a field at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        self.den[i][j][k]      += value*(1-di)*(1-dj)*(1-dk)
        self.den[i+1][j][k]    += value*(di)*(1-dj)*(1-dk)
        self.den[i+1][j+1][k]  += value*(di)*(dj)*(1-dk)
        self.den[i][j+1][k]    += value*(1-di)*(dj)*(1-dk)
        self.den[i][j][k+1]    += value*(1-di)*(1-dj)*(dk)
        self.den[i+1][j][k+1]  += value*(di)*(1-dj)*(dk)
        self.den[i+1][j+1][k+1]+= value*(di)*(dj)*(dk)
        self.den[i][j+1][k+1]  += value*(1-di)*(dj)*(dk)
    
    
    def scatter_den2(self, lc, value):
        """
        scatters scalar value onto a field at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        ----------
        """
        
        
        if lc [:,0].any() < 0 or \
             lc [:,0].any() >=self.worldObj.ni-1 or \
             lc [:,1].any() < 0 or \
             lc [:,1].any() >=self.worldObj.nj-1 or \
             lc [:,2].any() < 0 or \
             lc [:,2].any() >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = lc[:,0].astype(int)
        di = lc[:,0]-i

        j  = lc[:,1].astype(int)
        dj = lc[:,1]-j

        k  = lc[:,2].astype(int)
        dk = lc[:,2]-k
        
        self.den2 = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk, len(self.particleList)), np.dtype('f4')) 
        
        self.den2[i,j,k,:]      = value*(1-di)*(1-dj)*(1-dk)
        self.den2[i+1,j,k,:]    = value*(di)*(1-dj)*(1-dk)
        self.den2[i+1,j+1,k,:]  = value*(di)*(dj)*(1-dk)
        self.den2[i,j+1,k,:]    = value*(1-di)*(dj)*(1-dk)
        self.den2[i,j,k+1,:]    = value*(1-di)*(1-dj)*(dk)
        self.den2[i+1,j,k+1,:]  = value*(di)*(1-dj)*(dk)
        self.den2[i+1,j+1,k+1,:]= value*(di)*(dj)*(dk)
        self.den2[i,j+1,k+1,:]  = value*(1-di)*(dj)*(dk)
        
        for i in np.arange(0, self.worldObj.ni):
            for j in np.arange(0, self.worldObj.nj):
                for k in np.arange(0, self.worldObj.nk):
                    self.den[i,j,k] = np.sum(self.den2[i,j,k][:])
        
        
    
    
    def computeNumberDensity(self):
        """
        Compute particle number density

        Parameters
        ----------
        
        ----------
        """
        self.den = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk))
        
        for particle in self.particleList:
            lc = self.worldObj.XtoL(particle.pos)
            self.scatter_den(lc, self.mpw0)
            
        self.den = self.den / self.worldObj.node_vol
    
    def computeNumberDensity3(self):
        """
        Compute particle number density

        Parameters
        ----------
        
        ----------
        """
        self.den = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk))
        
        #for particle in self.particleList:
        
        pos_vec = np.zeros((len(self.particleList),3))
        
        for i in range(len(self.particleList)):
            pos_vec[i,:] = (self.particleList[i]).pos
        
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoLvec(pos_vec)
        
        for i in range(len(self.particleList)):
            self.scatter_den(lc[i,:], self.mpw)
            
        self.den = self.den / self.worldObj.node_vol
    
    
    def computeNumberDensity4(self):
        """
        Compute particle number density

        Parameters
        ----------
        
        ----------
        """
        self.den = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk))
        
        pos_vec2 = np.zeros((len(self.particleList),3))
        
        #for i in range(len(self.particleList)):
        #    pos_vec[i,:] = (self.particleList[i]).pos
        
        pos_vec2 = [(species1.particleList[i]).pos for i in range(len(species1.particleList))]
        
        pos_vec = np.asarray(pos_vec2)

        #get logical coordinate of particle's position
        lc = self.worldObj.XtoLvec(pos_vec)
        
        for i in range(len(self.particleList)):
            self.scatter_den(lc[i,:], self.mpw)
            
        self.den = self.den / self.worldObj.node_vol

    
    def computeNumberDensity2(self):
        """
        Compute particle number density

        Parameters
        ----------
        
        ----------
        """
        self.den = np.zeros((self.worldObj.ni, self.worldObj.nj, self.worldObj.nk))
        
        #for particle in self.particleList:
        
        pos_vec = np.zeros((len(self.particleList),3))
        
        for i in range(len(self.particleList)):
            pos_vec[i,:] = (self.particleList[i]).pos
        
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoLvec(pos_vec)

        self.scatter_den2(lc, self.mpw)

        self.den = self.den / self.worldObj.node_vol
        
    
    def gather_ef(self, lc):
        """
        gathers field value at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        data : numpy.ndarray
            electric field array
        ----------
        """
        
        if lc [0] < 0 or \
             lc [0] >=self.worldObj.ni-1 or \
             lc [1] < 0 or \
             lc [1] >=self.worldObj.nj-1 or \
             lc [2] < 0 or \
             lc [2] >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = int(lc[0])
        di = lc[0]-i

        j  = int(lc[1])
        dj = lc[1]-j

        k  = int(lc[2])
        dk = lc[2]-k

        # gather electric field onto particle position
         
        ef_x  = self.worldObj.ef[i][j][k][0]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][0]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][0]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][0]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][0]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][0]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][0]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][0]*(1-di)*(dj)*(dk)
        
        ef_y  = self.worldObj.ef[i][j][k][1]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][1]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][1]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][1]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][1]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][1]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][1]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][1]*(1-di)*(dj)*(dk)
        
        ef_z  = self.worldObj.ef[i][j][k][2]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j][k][2]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1][j+1][k][2]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j+1][k][2]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i][j][k+1][2]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j][k+1][2]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1][j+1][k+1][2]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i][j+1][k+1][2]*(1-di)*(dj)*(dk)
        
        ef_part = np.array([ef_x, ef_y, ef_z])
        
        return ef_part
    
    def gather_ef2(self, lc):
        """
        gathers field value at logical coordinate lc

        Parameters
        ----------
        lc : numpy.ndarray
            logical coordinate 
        data : numpy.ndarray
            electric field array
        ----------
        """
        
        if lc [:,0].any() < 0 or \
             lc [:,0].any() >=self.worldObj.ni-1 or \
             lc [:,1].any() < 0 or \
             lc [:,1].any() >=self.worldObj.nj-1 or \
             lc [:,2].any() < 0 or \
             lc [:,2].any() >=self.worldObj.nk-1:
            
            print("WARNING: point outside domain")
             
        i  = lc[:,0].astype(int)
        di = lc[:,0]-i

        j  = lc[:,1].astype(int)
        dj = lc[:,1]-j

        k  = lc[:,2].astype(int)
        dk = lc[:,2]-k

        # gather electric field onto particle position
        
        ef = np.zeros((len(lc[:,0]),3))
        
        ef[:,0]  = self.worldObj.ef[i,j,k][:,0]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1,j,k][:,0]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1,j+1,k][:,0]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i,j+1,k][:,0]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i,j,k+1][:,0]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1,j,k+1][:,0]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1,j+1,k+1][:,0]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i,j+1,k+1][:,0]*(1-di)*(dj)*(dk)
        
        ef[:,1]  = self.worldObj.ef[i,j,k][:,1]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1,j,k][:,1]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1,j+1,k][:,1]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i,j+1,k][:,1]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i,j,k+1][:,1]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1,j,k+1][:,1]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1,j+1,k+1][:,1]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i,j+1,k+1][:,1]*(1-di)*(dj)*(dk)
        
        ef[:,2]  = self.worldObj.ef[i,j,k][:,2]*(1-di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1,j,k][:,2]*(di)*(1-dj)*(1-dk)+\
                self.worldObj.ef[i+1,j+1,k][:,2]*(di)*(dj)*(1-dk)+\
                self.worldObj.ef[i,j+1,k][:,2]*(1-di)*(dj)*(1-dk)+\
                self.worldObj.ef[i,j,k+1][:,2]*(1-di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1,j,k+1][:,2]*(di)*(1-dj)*(dk)+\
                self.worldObj.ef[i+1,j+1,k+1][:,2]*(di)*(dj)*(dk)+\
                self.worldObj.ef[i,j+1,k+1][:,2]*(1-di)*(dj)*(dk)
        
        
        return ef
    
    
    def advance(self):
        #get the time step
        dt = self.worldObj.dt

        #save mesh bounds
        x0 = self.worldObj.x0
        xm = self.worldObj.xm

        # loop over all particles
        for particle in self.particleList:
        
            #get logical coordinate of particle's position
            lc = self.worldObj.XtoL(particle.pos)

            #electric field at particle position
            ef_part = self.gather_ef(lc)

            #update velocity from F=qE
            particle.vel += ef_part*(dt*self.charge/self.mass)

            #update position from v=dx/dt
            particle.pos += particle.vel*dt

            if self.worldObj.inSphere(particle.pos) or self.worldObj.inBounds(particle.pos)==False:
                particle.mpw = 0.0
                
        npp = len(self.particleList)

        p = 0

        while p < npp:
                if self.particleList[p].mpw == 0:
                    del self.particleList[p]

                    npp = npp - 1
                    p  = p - 1

                p = p+1
                    
    def advance2(self):
        #get the time step
        dt = self.worldObj.dt

        #save mesh bounds
        x0 = self.worldObj.x0
        xm = self.worldObj.xm

        # loop over all particles
        # for particle in self.particleList:
        
        pos_vec = np.zeros((len(self.particleList),3))
        
        for i in range(len(self.particleList)):
            pos_vec[i,:] = (self.particleList[i]).pos
        
        
        
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoLvec(pos_vec)

        #electric field at particle position
        ef = self.gather_ef2(lc)

        #update velocity from F=qE
        for i in range(len(self.particleList)):
            self.particleList[i].vel += ef[i,:]*(dt*self.charge/self.mass)
        
        #update position from v=dx/dt
        for i in range(len(self.particleList)):
            self.particleList[i].pos += self.particleList[i].vel*dt
        
        #did this particle leave the domain? reflect back
        for particle in self.particleList:
            for i in np.arange(3):

                if particle.pos[i] < x0[i]:
                    particle.pos[i]=2*x0[i]-particle.pos[i]
                    particle.vel[i]*=-1.0

                elif particle.pos[i] >= xm[i]:
                    particle.pos[i]=2*xm[i]-particle.pos[i]
                    particle.vel[i]*=-1.0
                    
    def advance3(self):
        #get the time step
        dt = self.worldObj.dt

        #save mesh bounds
        x0 = self.worldObj.x0
        xm = self.worldObj.xm

        # loop over all particles
        # for particle in self.particleList:
        
        pos_vec2 = np.zeros((len(self.particleList),3))
        
        #for i in range(len(self.particleList)):
        #    pos_vec[i,:] = (self.particleList[i]).pos
        
        pos_vec2 = [(species1.particleList[i]).pos for i in range(len(species1.particleList))]
        
        pos_vec = np.asarray(pos_vec2)

        
        #get logical coordinate of particle's position
        lc = self.worldObj.XtoLvec(pos_vec)

        #electric field at particle position
        ef = self.gather_ef2(lc)

        #update velocity from F=qE
        for i in range(len(self.particleList)):
            self.particleList[i].vel += ef[i,:]*(dt*self.charge/self.mass)
        
        #update position from v=dx/dt
        for i in range(len(self.particleList)):
            self.particleList[i].pos += self.particleList[i].vel*dt
        
        #is p

In [ ]:
world=World(21,21,41)
world.setTime(1E-7,400)
world.setExtents(-0.1, -0.1, 0.0 , 0.1, 0.1, 0.4)

world.addSphere(np.array([0.0,0.0,0.15]), 0.05, -100.0)
world.addInlet(0.0)
world.markBoundaryCells()

species1 = Species("O+", 16*world.AMU, world.QE, 1E2, world)

world.addSpeciesList([species1])

species1.computeNumberDensity()

world.computeChargeDensity()

world.potentialSolver8(1.4, 30000, 1E-4, 1E10, 0.0, 1.5)

world.efSolver2()

for i in range(1,401):
    
    print("ts = "+str(i)+", nO+: "+str(len(species1.particleList)))
    
    species1.ColdBeamSource(7000.0, 1E10)
    species1.advance()
    species1.computeNumberDensity()
    
    world.computeChargeDensity()
    
    world.potentialSolver8(1.4, 30000, 1E-4, 1E10, 0.0, 1.5)
    
    world.efSolver2()
    
    if i==1 or i%40==0:

        x_arr1 = np.zeros(len(species1.particleList))
        y_arr1 = np.zeros(len(species1.particleList))
        z_arr1 = np.zeros(len(species1.particleList))
        
        p_arr1 = np.zeros(len(species1.particleList))
        p_arr1 = np.zeros(len(species1.particleList))
        p_arr1 = np.zeros(len(species1.particleList))

        for j in np.arange(0,len(species1.particleList)):
            pos = species1.particleList[j].pos
            x_arr1[j] = pos[0]
            y_arr1[j] = pos[1]
            z_arr1[j] = pos[2]

        
        np.savetxt('1species_1_'+str(i).zfill(4)+'_x'+'.txt', x_arr1, fmt='%1.4f')
        np.savetxt('1species_1_'+str(i).zfill(4)+'_y'+'.txt', y_arr1, fmt='%1.4f')
        np.savetxt('1species_1_'+str(i).zfill(4)+'_z'+'.txt', z_arr1, fmt='%1.4f')

        np.save('1phi_'+str(i).zfill(4)+'.npy', world.phi)
        np.save('1rho_'+str(i).zfill(4)+'.npy', world.rho)
        np.save('1den_'+str(i).zfill(4)+'.npy', species1.den)

ts = 1, nO+: 0
ts = 2, nO+: 2800
ts = 3, nO+: 5600
ts = 4, nO+: 8400
ts = 5, nO+: 11200
ts = 6, nO+: 14000
ts = 7, nO+: 16800
ts = 8, nO+: 19600
ts = 9, nO+: 22400
ts = 10, nO+: 25200
ts = 11, nO+: 28000
ts = 12, nO+: 30800
ts = 13, nO+: 33600
ts = 14, nO+: 36400
ts = 15, nO+: 39199
ts = 16, nO+: 41999
ts = 17, nO+: 44798
ts = 18, nO+: 47598
ts = 19, nO+: 50398
ts = 20, nO+: 53198
ts = 21, nO+: 55998
ts = 22, nO+: 58798
ts = 23, nO+: 61598
ts = 24, nO+: 64398
ts = 25, nO+: 67198
ts = 26, nO+: 69997
ts = 27, nO+: 72797
ts = 28, nO+: 75597
ts = 29, nO+: 78397
ts = 30, nO+: 81196
ts = 31, nO+: 83995
ts = 32, nO+: 86795
ts = 33, nO+: 89592
ts = 34, nO+: 92390
ts = 35, nO+: 95188
ts = 36, nO+: 97986
ts = 37, nO+: 100783
ts = 38, nO+: 103580
ts = 39, nO+: 106376
ts = 40, nO+: 109173
ts = 41, nO+: 111966
ts = 42, nO+: 114761
ts = 43, nO+: 117557
ts = 44, nO+: 120353
ts = 45, nO+: 123151
ts = 46, nO+: 125945
ts = 47, nO+: 128741
ts = 48, nO+: 131532
ts = 49, nO+: 134322
ts = 50, nO+: 137115
ts